### Setup Environment

In [1]:
import os
import mlrun

> 2024-05-09 14:31:10,990 [warning] Error during request handling, retrying: {'exc': "HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v1/client-spec (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fff6e804460>: Failed to establish a new connection: [Errno 111] Connection refused'))", 'retry_count': 0, 'url': 'http://localhost:8080/api/v1/client-spec', 'method': 'GET'}
> 2024-05-09 14:31:18,029 [warning] Error during request handling, retrying: {'exc': "HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v1/client-spec (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fff6e804fa0>: Failed to establish a new connection: [Errno 111] Connection refused'))", 'retry_count': 1, 'url': 'http://localhost:8080/api/v1/client-spec', 'method': 'GET'}
> 2024-05-09 14:31:25,054 [warning] Error during request handling, retrying: {'exc': "HTTPConnectionPool(host='localhost',

In [2]:
# Set the environment:
mlrun.set_environment(env_file='mlrun.env', artifact_path='s3://mlrun/simple-demo')
#mlrun.set_environment('http://mlrun-api:8080', env_file='mlrun.env', artifact_path='s3://mlrun')
#mlrun.set_environment('http://mlrun-api:8080', artifact_path='./')

('default', 's3://mlrun')

### Create Project

In [3]:
# Create the project:
project_name='simple-test'
project_dir = os.path.abspath('./')
project = mlrun.get_or_create_project(project_name, project_dir, user_project=False)
print(project_dir)

> 2024-05-09 14:31:32,168 [info] Project loaded successfully: {'project_name': 'simple-test'}
/home/jovyan/simple_demo


### Create Training Function

In [4]:
# Create the training function.
trainer = project.set_function(
    "simple_serverless_function.py", name="trainer", kind="job", 
    image="mlrun/mlrun", 
    requirements=["minio"], # Using a requirement.txt file does not work. This also does not work if you run locally.
    handler="train_model"
)

### Run the function

In [5]:
# Sample hyperparameters
training_parameters = {
    'batch_size': 32,
    'device': 'cpu',
    'dropout_input': 0.2,
    'dropout_hidden': 0.5,
    'epochs': 5,
    'input_size': 784,
    'hidden_sizes': [1024, 1024, 1024, 1024],
    'lr': 0.025,
    'momentum': 0.5,
    'output_size': 10,
    'smoke_test_size': -1
    }

In [6]:
# Run the function locally on the Jupyter server.
trainer_run = project.run_function(
    "trainer",
    inputs={"data_bucket": "mnist"},
    params={"training_parameters": training_parameters},
    local=True
)

> 2024-05-09 14:31:32,194 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-09 14:31:32,195 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-09 14:31:32,205 [info] Storing function: {'name': 'trainer-train-model', 'uid': '46e3360cb4984c0d88329b8395635ffa', 'db': 'http://mlrun-api:8080'}
> 2024-05-09 14:31:32,239 [debug] Starting local run: /home/jovyan/simple_demo/simple_serverless_function.py # train_model
> 2024-05-09 14:31:32,378 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-05-09 14:31:32,379 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-09 14:31:32,379 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-0

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
simple-test,...95635ffa,0,May 09 14:31:32,completed,trainer-train-model,kind=localowner=jovyanhost=e9436980ce9c,data_bucket,"training_parameters={'batch_size': 32, 'device': 'cpu', 'dropout_input': 0.2, 'dropout_hidden': 0.5, 'epochs': 5, 'input_size': 784, 'hidden_sizes': [1024, 1024, 1024, 1024], 'lr': 0.025, 'momentum': 0.5, 'output_size': 10, 'smoke_test_size': -1}",,


> 2024-05-09 14:31:32,643 [info] Run execution finished: {'status': 'completed', 'name': 'trainer-train-model'}


In [7]:
#trainer.apply(mlrun.platforms.auto_mount())
trainer_run = project.run_function(
    "trainer",
    inputs={"data_bucket": "mnist"},
    params={"training_parameters": training_parameters},
    local=True
)

> 2024-05-09 14:31:32,649 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-05-09 14:31:32,658 [info] Storing function: {'name': 'trainer-train-model', 'uid': 'e86b6a1e2e164cdbaa14ba1c793b9672', 'db': 'http://mlrun-api:8080'}
> 2024-05-09 14:31:32,686 [debug] Starting local run: /home/jovyan/simple_demo/simple_serverless_function.py # train_model
> 2024-05-09 14:31:32,702 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-05-09 14:31:32,703 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-09 14:31:32,704 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-05-09 14:31:32,705 [debug] The packagers manager collected the packager: DictPackager(packable_type=

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
simple-test,...793b9672,0,May 09 14:31:32,completed,trainer-train-model,kind=localowner=jovyanhost=e9436980ce9c,data_bucket,"training_parameters={'batch_size': 32, 'device': 'cpu', 'dropout_input': 0.2, 'dropout_hidden': 0.5, 'epochs': 5, 'input_size': 784, 'hidden_sizes': [1024, 1024, 1024, 1024], 'lr': 0.025, 'momentum': 0.5, 'output_size': 10, 'smoke_test_size': -1}",,


> 2024-05-09 14:31:32,788 [info] Run execution finished: {'status': 'completed', 'name': 'trainer-train-model'}


In [8]:
project.save()